In [ ]:
# ================================================
# 1. IMPORT LIBRARIES
# ================================================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ================================================
# 2. LOAD DATA
# ================================================
train = pd.read_csv("/kaggle/input/mock-test-2-mse-2/train.csv")
test  = pd.read_csv("/kaggle/input/mock-test-2-mse-2/test.csv")

# Look at the data
print(train.head())
print(train.info())
print(train.isnull().sum())
print(train['Status'].value_counts())

# ================================================
# 3. SPLIT FEATURES & TARGET
# ================================================
y = train["Status"]              # Target column
X = train.drop("Status", axis=1) # Input features

# Separate numeric & categorical columns
num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

# ================================================
# 4. HANDLE MISSING VALUES
# ================================================
# Numeric -> fill with median
X[num_cols] = X[num_cols].fillna(X[num_cols].median())
test[num_cols] = test[num_cols].fillna(test[num_cols].median())

# Categorical -> fill with mode (most frequent value)
X[cat_cols] = X[cat_cols].fillna(X[cat_cols].mode().iloc[0])
test[cat_cols] = test[cat_cols].fillna(test[cat_cols].mode().iloc[0])

# Reset index (safety)
X = X.reset_index(drop=True)
test = test.reset_index(drop=True)

# ================================================
# 5. LABEL ENCODE TARGET (C, CL, D → 0,1,2)
# ================================================
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Label Mapping:", le.classes_)
# Output → ['C', 'CL', 'D']

# ================================================
# 6. ONE-HOT ENCODE + RANDOM FOREST MODEL
# ================================================
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Preprocessing for categorical & numeric cols
preprocess = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ("num", "passthrough", num_cols)
])

# Build a pipeline
model = Pipeline([
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(random_state=42))
])

# ================================================
# 7. TRAIN / VALIDATION SPLIT
# ================================================
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# ================================================
# 8. TRAIN THE MODEL
# ================================================
model.fit(X_train, y_train)

# ================================================
# 9. MODEL EVALUATION
# ================================================
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

y_pred = model.predict(X_val)
y_prob = model.predict_proba(X_val) # probability for AUC

print("Accuracy :", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred, average='macro'))
print("Recall   :", recall_score(y_val, y_pred, average='macro'))
print("F1 Score :", f1_score(y_val, y_pred, average='macro'))
print("ROC AUC  :", roc_auc_score(y_val, y_prob, multi_class='ovr'))

# ================================================
# 10. TRAIN ON FULL DATASET
# ================================================
model.fit(X, y_encoded)

# ================================================
# 11. PREDICT TEST SET
# ================================================
test_pred = model.predict_proba(test)

# ================================================
# 12. CREATE SUBMISSION FILE
# ================================================
submission = pd.DataFrame({
    "id": test["id"],
    "Status_C":  test_pred[:, le.transform(['C'])[0]],
    "Status_CL": test_pred[:, le.transform(['CL'])[0]],
    "Status_D":  test_pred[:, le.transform(['D'])[0]],
})

print("Duplicate IDs:", submission["id"].duplicated().sum())

submission.to_csv("submission6.csv", index=False)
print("submission6.csv CREATED!")

submission.head()


In [ ]:
# ============================================
# 1. IMPORT LIBRARIES
# ============================================
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# For clean output
import warnings
warnings.filterwarnings("ignore")

# ============================================
# 2. LOAD DATA
# ============================================
df = pd.read_csv("/content/drive/MyDrive/Titanic-Dataset.csv")

print("First 5 rows:")
print(df.head())

print("\nDataset Info:")
print(df.info())

# Remove rows where target is missing
df = df.dropna(subset=['Survived'])

# ============================================
# 3. EDA (GRAPHS)
# ============================================

# --- Missing Values Plot ---
plt.figure(figsize=(10,4))
df.isnull().sum().plot(kind='bar', color='red')
plt.title("Missing Values Per Column")
plt.show()

# --- Survival Count ---
plt.figure(figsize=(6,4))
sns.countplot(x=df['Survived'])
plt.title("Survival Count (0=No, 1=Yes)")
plt.show()

# --- Survival by Gender ---
plt.figure(figsize=(6,4))
sns.countplot(x='Sex', hue='Survived', data=df)
plt.title("Survival by Gender")
plt.show()

# --- Age Distribution ---
plt.figure(figsize=(8,4))
sns.histplot(df['Age'], kde=True)
plt.title("Age Distribution")
plt.show()

# --- Correlation Heatmap ---
plt.figure(figsize=(10,6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

# ============================================
# 4. SELECT FEATURES FOR PREDICTION
# ============================================
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# ============================================
# 5. HANDLE MISSING VALUES
# ============================================

# Replace missing Age with median
X['Age'].fillna(X['Age'].median(), inplace=True)

# Replace missing Embarked with most frequent
X['Embarked'].fillna(X['Embarked'].mode()[0], inplace=True)

# ============================================
# 6. LABEL ENCODING (text → numbers)
# ============================================
le = LabelEncoder()

X['Sex'] = le.fit_transform(X['Sex'])         # male/female → 0/1
X['Embarked'] = le.fit_transform(X['Embarked'])  # S, C, Q → 0/1/2

# ============================================
# 7. TRAIN/TEST SPLIT
# ============================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ============================================
# 8. TRAIN RANDOM FOREST MODEL
# ============================================
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)

# ============================================
# 9. PREDICT ON TEST DATA
# ============================================
y_pred = model.predict(X_test)

# ============================================
# 10. EVALUATE MODEL
# ============================================
acc = accuracy_score(y_test, y_pred)
print("\n✅ Accuracy:", round(acc * 100, 2), "%")

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# ============================================
# 11. SAMPLE PREDICTION
# ============================================
sample = X_test.iloc[0:1]
print("\nSample Passenger:")
print(sample)

prediction = model.predict(sample)[0]
print("\nPredicted (1 = Survived):", prediction)


In [ ]:
# ===============================================
# 1. IMPORT LIBRARIES
# ===============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix

# ===============================================
# 2. LOAD DATA
# ===============================================
train_df = pd.read_csv('/Users/piyushagarwal/Documents/Python Files/Mock Dataset/train (1).csv')
test_df  = pd.read_csv('/Users/piyushagarwal/Documents/Python Files/Mock Dataset/test (1).csv')

train_df.info()
train_df.head()

# ===============================================
# 3. DROP USELESS COLUMNS
# (ID columns don't help in ML)
# ===============================================
train_df.drop(['id','CustomerId','Surname'], axis=1, inplace=True)

train_df.head()

# ===============================================
# 4. CHECK BASIC DATA QUALITY
# ===============================================
train_df.isnull().sum()
train_df.duplicated().sum()
train_df.nunique()

# Target and categorical value counts
train_df['Geography'].value_counts()
train_df['Gender'].value_counts()
train_df['NumOfProducts'].value_counts()
train_df['HasCrCard'].value_counts()
train_df['IsActiveMember'].value_counts()
train_df['Exited'].value_counts()

# ===============================================
# 5. ONE-HOT ENCODING (Convert categories → numbers)
# drop_first removes dummy trap
# ===============================================
df_encoded = pd.get_dummies(train_df, columns=['Gender', 'Geography'],
                            drop_first=True, dtype=int)
df_encoded.info()

train_df = df_encoded

# ===============================================
# 6. BOXPLOTS BEFORE OUTLIER REMOVAL
# Check numeric columns vs target
# ===============================================
category_col = 'Exited'
numeric_cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']

for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=category_col, y=col, data=train_df)
    plt.title(f'Boxplot of {col} by {category_col}')
    plt.show()

# ===============================================
# 7. REMOVE OUTLIERS USING IQR
# ===============================================
def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower) & (df[col] <= upper)]
    return df

df_clean = remove_outliers_iqr(train_df, numeric_cols)
print("Before:", train_df.shape[0], "rows")
print("After :", df_clean.shape[0], "rows")

# Boxplots after outlier removal
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=category_col, y=col, data=df_clean)
    plt.title(f'Boxplot of {col} by {category_col}')
    plt.show()

# ===============================================
# 8. CORRELATION HEATMAP
# ===============================================
corr = train_df.corr(numeric_only=True)
plt.figure(figsize=(10,6))
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap", fontsize=14)
plt.show()

# ===============================================
# 9. PAIRPLOT (Multivariate EDA)
# ===============================================
sns.pairplot(train_df, vars=['CreditScore', 'Balance', 'Age',
                             'EstimatedSalary'],
             hue='Exited', diag_kind='kde')
plt.show()

# ===============================================
# 10. DEFINE FEATURES & TARGET
# ===============================================
X = train_df.drop('Exited', axis=1)
y = train_df['Exited']

# ===============================================
# 11. BALANCE DATA USING SMOTE
# (Fix Imbalanced Target)
# ===============================================
from imblearn.over_sampling import SMOTE

unique, counts = np.unique(y, return_counts=True)
min_class_size = counts.min()
k = min(5, max(1, min_class_size - 1))  # dynamic k

smote = SMOTE(random_state=42, k_neighbors=k)
X_res, y_res = smote.fit_resample(X, y)

print("Before SMOTE:", dict(zip(unique, counts)))
print("After SMOTE :", dict(zip(*np.unique(y_res, return_counts=True))))

# ===============================================
# 12. STANDARD SCALING
# ===============================================
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

# ===============================================
# 13. TRAIN–VALIDATION SPLIT
# ===============================================
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y_res, test_size=0.2, random_state=42, stratify=y_res
)

# ===============================================
# 14. LOGISTIC REGRESSION MODEL
# ===============================================
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

# Metrics
print("\nLogistic Regression Results")
print("Accuracy :", accuracy_score(y_val, y_pred))
print("ROC-AUC :", roc_auc_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# ===============================================
# 15. SVM (Using Tuned Best Parameters)
# ===============================================
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=10, gamma=0.1, coef0=0.0)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

print("\nSVC Results")
print("Accuracy :", accuracy_score(y_val, y_pred))
print("ROC-AUC :", roc_auc_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# ===============================================
# 16. RANDOM FOREST (Best Tuned Parameters)
# ===============================================
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt'
)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

print("\nRandom Forest Results")
print("Accuracy :", accuracy_score(y_val, y_pred))
print("ROC-AUC :", roc_auc_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# ===============================================
# 17. TRAIN FINAL MODEL FOR PREDICTION
# ===============================================
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=30,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    oob_score=True,
    random_state=42
)

rf.fit(X_scaled, y_res)   # Train on full resampled dataset

# ===============================================
# 18. PROCESS TEST DATA
# ===============================================
test_df.info()

test_ids = test_df['id']
test_df.drop(['id','CustomerId','Surname'], axis=1, inplace=True)

test_df.duplicated().sum()

# Same encoding as training
test_df = pd.get_dummies(test_df, columns=['Gender', 'Geography'],
                         drop_first=True, dtype=int)

# Scale test data
X_test = scaler.transform(test_df)

# ===============================================
# 19. PREDICT TEST PROBABILITIES
# ===============================================
y_pred = rf.predict_proba(X_test)

# ===============================================
# 20. CREATE SUBMISSION FILE
# ===============================================
sub_df = pd.DataFrame({
    'id': test_ids,
    'Exited': y_pred[:,1]   # probability of exit
})

sub_df.to_csv('submission_Bank.csv', index=False, float_format='%.6f')

print("\nSubmission File Created Successfully!")


In [ ]:
# ======================================================
# 1) IMPORT LIBRARIES
# ======================================================
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    classification_report, roc_curve
)
import joblib
import warnings

warnings.filterwarnings("ignore")
np.random.seed(42)

# ======================================================
# 2) LOAD DATA (auto-detect CSV inside folder)
# ======================================================
input_path = "/kaggle/input/health-heart-disease-data"

# If path is directory → pick first CSV
if os.path.isdir(input_path):
    csv_file = glob.glob(os.path.join(input_path, "*.csv"))[0]
else:
    csv_file = input_path

print("Reading:", csv_file)
df = pd.read_csv(csv_file)

# ======================================================
# 3) BASIC EDA – preview data
# ======================================================
print("\n--- Data Head ---")
display(df.head())

print("\n--- Describe ---")
display(df.describe().T)

print("\nTarget distribution:")
print(df['disease_risk'].value_counts())

# ======================================================
# 4) BOXPLOTS (Before Imputation)
# ======================================================
num_cols = df.select_dtypes(include=np.number).columns.tolist()
num_cols.remove('disease_risk')

plt.figure(figsize=(10, len(num_cols)*2))
for i, col in enumerate(num_cols, 1):
    plt.subplot(len(num_cols), 1, i)
    plt.boxplot(df[col].dropna(), vert=False)
    plt.title(f"{col} (before imputation)")
plt.tight_layout()
plt.show()

# ======================================================
# 5) HANDLE MISSING VALUES
#    Numeric → median, Categorical → mode
# ======================================================
df_imputed = df.copy()

for col in df_imputed.columns:
    if df_imputed[col].isnull().sum() > 0:
        if pd.api.types.is_numeric_dtype(df_imputed[col]):
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
        else:
            df_imputed[col].fillna(df_imputed[col].mode()[0], inplace=True)

# ======================================================
# 6) BOXPLOTS (After Imputation, Before Outliers)
# ======================================================
plt.figure(figsize=(10, len(num_cols)*2))
for i, col in enumerate(num_cols, 1):
    plt.subplot(len(num_cols), 1, i)
    plt.boxplot(df_imputed[col], vert=False)
    plt.title(f"{col} (after imputation)")
plt.tight_layout()
plt.show()

# ======================================================
# 7) OUTLIER REMOVAL USING IQR
# ======================================================
df_no_out = df_imputed.copy()

for col in num_cols:
    Q1 = df_no_out[col].quantile(0.25)
    Q3 = df_no_out[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df_no_out = df_no_out[(df_no_out[col] >= lower) & (df_no_out[col] <= upper)]

# ======================================================
# 8) BOXPLOTS (After Outlier Removal)
# ======================================================
plt.figure(figsize=(10, len(num_cols)*2))
for i, col in enumerate(num_cols, 1):
    plt.subplot(len(num_cols), 1, i)
    plt.boxplot(df_no_out[col], vert=False)
    plt.title(f"{col} (after outlier removal)")
plt.tight_layout()
plt.show()

# Save cleaned data
df_no_out.to_csv("/kaggle/working/health_data_cleaned.csv", index=False)

# ======================================================
# 9) TRAIN-TEST SPLIT + SCALING
# ======================================================
X = df_no_out.drop('disease_risk', axis=1)
y = df_no_out['disease_risk']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ======================================================
# 10) RANDOM FOREST + RandomizedSearchCV
# ======================================================
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': [100,150,200,250,300],
    'max_depth': [5,8,10,None],
    'min_samples_split': [2,3,4,5,6,8,10],
    'min_samples_leaf': [1,2,3,4],
    'bootstrap': [True, False]
}

rand_search = RandomizedSearchCV(
    rf, param_dist, n_iter=20, cv=3,
    n_jobs=-1, random_state=42, scoring='f1', verbose=1
)

rand_search.fit(X_train_scaled, y_train)
best_rf = rand_search.best_estimator_

# ======================================================
# 11) TEST SET EVALUATION
# ======================================================
y_pred = best_rf.predict(X_test_scaled)
y_proba = best_rf.predict_proba(X_test_scaled)[:,1]

print("\n--- Test Metrics ---")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))
print("AUC      :", roc_auc_score(y_test, y_proba))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix Plot
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5,4))
plt.imshow(cm)
plt.title("Confusion Matrix")
plt.colorbar()
plt.show()

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],'--')
plt.title("ROC Curve")
plt.show()

# Feature Importance
feat_imp = pd.Series(best_rf.feature_importances_, index=X.columns)
display(feat_imp.sort_values(ascending=False))

# ======================================================
# 12) SAVE FINAL MODEL + SCALER
# ======================================================
joblib.dump(best_rf, "/kaggle/working/rf_best_model.joblib")
joblib.dump(scaler, "/kaggle/working/scaler.joblib")


In [ ]:
# ============================================================
#   COMPLETE ONE-CELL SOLUTION WITH COMMENTS + GRAPHS
# ============================================================

# -------------------- IMPORTS --------------------
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error


# -------------------- LOAD DATA --------------------
df = pd.read_csv('/kaggle/input/ai-201-b-minor-mse-1/train_data.csv')
df.drop_duplicates(inplace=True)


# -------------------- OUTLIER REMOVAL --------------------
z_scores = (df - df.mean()) / df.std()
df_clean = df[(np.abs(z_scores) <= 3).all(axis=1)]
df = df_clean.copy()


# -------------------- DATA VISUALIZATION --------------------
plt.figure(figsize=(14, 6))
for i, col in enumerate(df.columns[:-1]):
    plt.subplot(1, 3, i+1)
    sns.histplot(df[col], kde=True, bins=25)
    plt.title(f"Distribution of {col}")
plt.tight_layout()
plt.show()

plt.figure(figsize=(6, 4))
sns.histplot(df['Product_Strength_Deviation'], kde=True, bins=30, color='orange')
plt.title("Distribution of Target")
plt.show()

plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()


# -------------------- MISSING VALUE HANDLING --------------------
df['Machine_Speed_Deviation'] = df['Machine_Speed_Deviation'].fillna(df['Machine_Speed_Deviation'].mean())
df['Pressure_Level_Deviation'] = df['Pressure_Level_Deviation'].fillna(df['Pressure_Level_Deviation'].mean())
df['Temperature_Deviation'] = df['Temperature_Deviation'].fillna(df['Temperature_Deviation'].median())
df['Product_Strength_Deviation'] = df['Product_Strength_Deviation'].fillna(df['Product_Strength_Deviation'].mean())


# -------------------- TRAIN-TEST SPLIT --------------------
X = df.drop('Product_Strength_Deviation', axis=1)
y = df['Product_Strength_Deviation']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


# -------------------- LINEAR REGRESSION --------------------
lr = LinearRegression()
lr.fit(X_scaled, y)
y_pred_lr = lr.predict(X_val)

print("Linear Regression R2:", r2_score(y_val, y_pred_lr))
print("Linear Regression MSE:", mean_squared_error(y_val, y_pred_lr))


# -------------------- POLYNOMIAL REGRESSION --------------------
degree = 7
poly = PolynomialFeatures(degree=degree)
X_poly = poly.fit_transform(X_scaled)

X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(
    X_poly, y, test_size=0.1, random_state=42
)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train_poly)

y_pred_poly = poly_model.predict(X_test_poly)

print(f"Polynomial Regression (Degree={degree}) R2:", r2_score(y_test_poly, y_pred_poly))
print(f"Polynomial Regression (Degree={degree}) MSE:", mean_squared_error(y_test_poly, y_pred_poly))


# -------------------- DECISION TREE --------------------
dt = DecisionTreeRegressor(random_state=42)
param_grid_dt = {'max_depth': [2,4,6,8,10]}

grid_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='r2')
grid_dt.fit(X_scaled, y)

y_pred_dt = grid_dt.predict(X_val)

print("Decision Tree R2:", r2_score(y_val, y_pred_dt))
print("Decision Tree MSE:", mean_squared_error(y_val, y_pred_dt))
print("Best DT Params:", grid_dt.best_params_)


# -------------------- KNN REGRESSION --------------------
knn = KNeighborsRegressor()
param_grid_knn = {
    'n_neighbors': [3,5,7,9,11],
    'weights': ['uniform','distance'],
    'p': [1,2]
}

grid_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='r2')
grid_knn.fit(X_train, y_train)

y_pred_knn = grid_knn.predict(X_val)

print("KNN R2:", r2_score(y_val, y_pred_knn))
print("KNN MSE:", mean_squared_error(y_val, y_pred_knn))
print("Best KNN Params:", grid_knn.best_params_)


# -------------------- PREDICT ON TEST FILE --------------------
df_test = pd.read_csv('/kaggle/input/ai-201-b-minor-mse-1/test_data.csv')
X_test = df_test.drop(columns=['id'])

X_test_poly = poly.transform(scaler.transform(X_test))
y_new_pred = poly_model.predict(X_test_poly)

submission = pd.DataFrame({
    'id': df_test['id'],
    'Product_Strength_Deviation': y_new_pred
})

submission.to_csv("submission.csv", index=False)
submission.head()


In [ ]:
# ==============================================
# 📦 Multilinear Regression | Baseline Notebook
# Evaluation: MSE & R² Score
# ==============================================

# --- 1. Import Libraries ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


# --- 2. Load Data ---
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')


# --- 3. Quick Look ---
print("Shape:", train.shape)
print("\nFirst 5 rows:\n", train.head())


# --- 4. Basic Info ---
print("\nData Info:")
print(train.info())

print("\nSummary Statistics:")
print(train.describe())


# --- 5. Correlation Heatmap (to see feature relationships) ---
plt.figure(figsize=(6,5))
sns.heatmap(train.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


# --- 6. Pairplot (visualize patterns between features) ---
sns.pairplot(train[['size','rooms','floor','age','price']], diag_kind='hist')
plt.suptitle("Pairwise Feature Relationships", y=1.02)
plt.show()


# --- 7. Prepare Data for Training ---
# Independent variables (features)
X = train[['size','rooms','floor','age']]

# Target variable
y = train['price']


# --- 8. Train Linear Regression Model ---
model = LinearRegression()
model.fit(X, y)


# --- 9. Evaluate Model on Training Data ---
y_pred = model.predict(X)

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("\n✅ Model Evaluation on Training Data:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.4f}")


# --- 10. Check Coefficients ---
coeff_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})

print("\nModel Coefficients:")
print(coeff_df)
print(f"Intercept: {model.intercept_:.2f}")


# --- 11. Predict on Test Data ---
test_pred = model.predict(test[['size','rooms','floor','age']])


# --- 12. Create Submission File ---
submission = pd.DataFrame({
    'id': test['id'],
    'price': test_pred.round(2)     # round to 2 decimals
})

submission.to_csv('submission.csv', index=False)

print("\n✅ Clean Submission Preview:")
print(submission.to_string(index=False))


# --- 13. Optional: Actual vs Predicted Plot ---
plt.figure(figsize=(6,4))
plt.scatter(y, y_pred, alpha=0.7, color='green')
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.title("Actual vs Predicted Price")
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# ======================================================================
# 📌 COMPLETE MACHINE LEARNING PIPELINE (RF + HyperTuning + EDA)
# ======================================================================

# 1️⃣ IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

# ======================================================================
# 2️⃣ HOW TO CREATE A CSV FILE (Example)
# ======================================================================
data = {
    "Pclass": [1, 3, 2],
    "Sex": ["male", "female", "male"],
    "Age": [22, 28, np.nan],
    "Fare": [71.2, 7.25, 12.85],
    "Survived": [0, 1, 1]
}

demo_df = pd.DataFrame(data)
demo_df.to_csv("sample.csv", index=False)
print("Sample CSV created: sample.csv\n")

# ======================================================================
# 3️⃣ LOAD DATA
# ======================================================================
df = pd.read_csv("/content/drive/MyDrive/Titanic-Dataset.csv")

print("First 5 rows:")
print(df.head())

# ======================================================================
# 4️⃣ REMOVE NULL VALUES (multiple ways)
# ======================================================================

# ❗ Remove rows where target is null
df = df.dropna(subset=['Survived'])

# ❗ Fill numeric missing values with median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# ❗ Fill categorical missing values with mode
cat_cols = df.select_dtypes(include=['object']).columns
df[cat_cols] = df[cat_cols].fillna(df[cat_cols].mode().iloc[0])

print("\nNull values removed.\n")

# ======================================================================
# 5️⃣ EDA → DISTRIBUTION PLOTS
# ======================================================================
plt.figure(figsize=(6,4))
sns.histplot(df['Age'], kde=True)
plt.title("Age Distribution")
plt.show()

plt.figure(figsize=(6,4))
sns.countplot(data=df, x='Survived')
plt.title("Survived Count")
plt.show()

# Correlation heatmap
plt.figure(figsize=(7,6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

# ======================================================================
# 6️⃣ OUTLIER CAPPING (YOUR CODE) — CATEGORY WISE
# ======================================================================

def cap_outliers_categorywise_all(df, cat_col, num_cols):
    def cap_group(group):
        group = group.copy()
        for col in num_cols:
            Q1 = group[col].quantile(0.25)
            Q3 = group[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            group[col] = group[col].clip(lower, upper)  # Cap values
        return group

    df_capped = df.groupby(cat_col, group_keys=False, observed=True, sort=False).apply(cap_group)
    return df_capped

# Apply outlier capping using "Pclass" as category
numeric_cols = ['Age', 'Fare']
df = cap_outliers_categorywise_all(df, 'Pclass', numeric_cols)

print("\nOutliers capped category-wise.\n")

# ======================================================================
# 7️⃣ LABEL ENCODING (with INVERSE TRANSFORM support)
# ======================================================================

le_sex = LabelEncoder()
le_embarked = LabelEncoder()

df['Sex'] = le_sex.fit_transform(df['Sex'])
df['Embarked'] = le_embarked.fit_transform(df['Embarked'])

print("\nLabel Encoding Completed.\n")

# To decode later:
# original_value = le_sex.inverse_transform([encoded_value])

# ======================================================================
# 8️⃣ FEATURE SELECTION
# ======================================================================

features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
X = df[features]
y = df['Survived']

# ======================================================================
# 9️⃣ TRAIN/TEST SPLIT
# ======================================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# ======================================================================
# 🔟 RANDOM FOREST (BASE MODEL)
# ======================================================================
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# ======================================================================
# 1️⃣1️⃣ HYPERPARAMETER TUNING — GRID SEARCH
# ======================================================================

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=3,
    scoring='accuracy',
    n_jobs=-1
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_

print("\nBest Parameters:", grid.best_params_)

# ======================================================================
# 1️⃣2️⃣ PREDICT
# ======================================================================
y_pred = best_model.predict(X_test)
y_prob = best_model.predict_proba(X_test)[:,1]  # For AUC

# ======================================================================
# 1️⃣3️⃣ EVALUATION METRICS
# ======================================================================

print("\n================= MODEL METRICS =================")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred))
print("F1 Score :", f1_score(y_test, y_pred))
print("ROC-AUC  :", roc_auc_score(y_test, y_prob))

print("\nClassification Report:\n", classification_report(y_test, y_pred))

print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

# ======================================================================
# 1️⃣4️⃣ INVERSE TRANSFORM EXAMPLE
# ======================================================================
encoded_val = df['Sex'].iloc[0]
decoded_val = le_sex.inverse_transform([encoded_val])[0]

print("\nEncoded Sex =", encoded_val, " → Decoded =", decoded_val)


In [ ]:
# =====================================================================
# 📌 COMPLETE PIPELINE WITH OUTLIER ANALYSIS + ONE-HOT + RANDOM FOREST
# =====================================================================

# 1️⃣ Import Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss, confusion_matrix,
    classification_report
)
from sklearn.pipeline import Pipeline


# =====================================================================
# 2️⃣ Load Data
# =====================================================================
train = pd.read_csv("/kaggle/input/mock-test-2-mse-2/train.csv")
test = pd.read_csv("/kaggle/input/mock-test-2-mse-2/test.csv")

print(train.head())


# =====================================================================
# 3️⃣ Split Target + Features
# =====================================================================
y = train["Status"]
X = train.drop(columns=["Status"])


# =====================================================================
# 4️⃣ Handle Missing Values (Safe Method)
# =====================================================================
for col in X.columns:
    if X[col].dtype == 'object':
        # Categorical → fill with mode
        X[col] = X[col].fillna(X[col].mode()[0])
        test[col] = test[col].fillna(test[col].mode()[0])
    else:
        # Numerical → fill with mean
        X[col] = X[col].fillna(X[col].mean())
        test[col] = test[col].fillna(test[col].mean())


# =====================================================================
# 5️⃣ Outlier Analysis
# =====================================================================
print("\n================ OUTLIER ANALYSIS ================")

# Identify numeric columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns


# ------------------------- (A) Isolation Forest ------------------------
iso = IsolationForest(contamination=0.03, random_state=42)
outlier_flags = iso.fit_predict(X[numeric_cols])

# -1 = outlier, 1 = normal
X_clean = X[outlier_flags == 1]
y_clean = y[outlier_flags == 1]

print(f"Rows before IsolationForest: {len(X)}")
print(f"Rows after cleaning: {len(X_clean)}")


# ------------------------- (B) IQR Outlier Capping ----------------------
# You asked to include this function

def cap_outliers_categorywise_all(df, cat_col, num_cols):
    """
    Caps outliers within each categorical group using IQR method.
    """
    def cap_group(group):
        group = group.copy()
        for col in num_cols:
            Q1 = group[col].quantile(0.25)
            Q3 = group[col].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
            group[col] = group[col].clip(lower, upper)
        return group

    return df.groupby(cat_col, group_keys=False, observed=True, sort=False).apply(cap_group)

categorical_col_for_capping = "Region" if "Region" in X_clean.columns else X_clean.select_dtypes(include="object").columns[0]

X_capped = cap_outliers_categorywise_all(X_clean.join(y_clean), categorical_col_for_capping, numeric_cols)

# Separate target back
y_clean = X_capped["Status"]
X_clean = X_capped.drop(columns=["Status"])

print("Outlier capping complete!")


# =====================================================================
# 6️⃣ Train–Validation Split (after outlier cleaning)
# =====================================================================
X_train, X_val, y_train, y_val = train_test_split(
    X_clean, y_clean, test_size=0.2, random_state=42, stratify=y_clean
)


# =====================================================================
# 7️⃣ Preprocessing (OneHotEncoding for categorical)
# =====================================================================
categorical_cols = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ("num", "passthrough", numeric_cols)
    ]
)


# =====================================================================
# 8️⃣ RF Pipeline
# =====================================================================
base_model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])


# =====================================================================
# 9️⃣ Hyperparameter Tuning
# =====================================================================
param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [10, 20, None],
    "classifier__min_samples_split": [2, 5],
    "classifier__min_samples_leaf": [1, 2]
}

grid = GridSearchCV(
    base_model,
    param_grid,
    scoring="neg_log_loss",
    cv=3,
    verbose=1,
    n_jobs=-1
)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_

print("\n🔎 Best Parameters:")
print(grid.best_params_)


# =====================================================================
# 🔟 Validation Results
# =====================================================================
y_pred = best_model.predict(X_val)
y_proba = best_model.predict_proba(X_val)

print("\n================ VALIDATION METRICS ================")
print("Accuracy :", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred, average="weighted"))
print("Recall   :", recall_score(y_val, y_pred, average="weighted"))
print("F1 Score :", f1_score(y_val, y_pred, average="weighted"))
print("Log Loss :", log_loss(y_val, y_proba))

try:
    print("AUC Score:", roc_auc_score(y_val, y_proba, multi_class="ovr"))
except:
    print("AUC Score: multiclass AUC failed")

print("\nConfusion Matrix:")
print(confusion_matrix(y_val, y_pred))


# =====================================================================
# 1️⃣1️⃣ Prepare Test Data (same columns as training)
# =====================================================================
test_processed = test[X.columns]   # aligns column order


# =====================================================================
# 1️⃣2️⃣ Submission File
# =====================================================================
predict_proba = best_model.predict_proba(test_processed)

class_labels = list(best_model.classes_)

submission = pd.DataFrame({"id": test["id"]})

for i, cls in enumerate(class_labels):
    submission[f"Status_{cls}"] = predict_proba[:, i]

submission.to_csv("Ayush.csv", index=False)

print("\n📁 Submission Preview:")
print(submission.head())
